In [2]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, models
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import mmcv, cv2
from PIL import Image, ImageDraw
from IPython import display

In [3]:
workers = 4

In [4]:
# from facenet_pytorch import MTCNN
# import torch
# import numpy as np
# import mmcv, cv2
# from PIL import Image, ImageDraw
# from IPython import display
# import ffmpeg


In [5]:
# display.Video('trimmed4.mp4', width=640)

In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'Running on device: {device}')

Running on device: cuda:0


In [7]:
mtcnn = MTCNN(keep_all=True, device=device)

In [52]:
# def collate_fn(x):
#     return x[0]

# dataset = datasets.ImageFolder('./data/images')
# dataset.idx_to_class = {i:c for c, i in dataset.class_to_idx.items()}
# loader = DataLoader(dataset, collate_fn=collate_fn, num_workers=workers)



data_transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.Resize((110, 75)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
face_dataset = datasets.ImageFolder('./data/images', transform=data_transform)
dataset_loader = torch.utils.data.DataLoader(face_dataset, shuffle=True, num_workers=4)

In [53]:
for tensor, idx_label in face_dataset:
    display.display(tensor)

tensor([[[-1.8439, -1.8439, -1.8439,  ..., -1.9809, -1.5699, -0.8335],
         [-1.8439, -1.8439, -1.8439,  ..., -2.0837, -1.7069, -1.0219],
         [-1.8439, -1.8439, -1.8439,  ..., -2.1008, -1.8953, -1.4500],
         ...,
         [-2.1179, -2.1179, -2.1179,  ...,  0.4679,  0.3994,  0.3309],
         [-2.1179, -2.1179, -2.1179,  ...,  0.4508,  0.3994,  0.3309],
         [-2.1179, -2.1179, -2.1179,  ...,  0.4337,  0.3823,  0.3309]],

        [[-1.7556, -1.7556, -1.7556,  ..., -1.8782, -1.4755, -0.7227],
         [-1.7556, -1.7556, -1.7556,  ..., -1.9832, -1.5980, -0.9153],
         [-1.7556, -1.7556, -1.7556,  ..., -2.0007, -1.7731, -1.3179],
         ...,
         [-1.9657, -1.9657, -1.9657,  ..., -1.9657, -2.0007, -2.0357],
         [-1.9657, -1.9657, -1.9657,  ..., -1.9832, -2.0007, -2.0357],
         [-1.9657, -1.9657, -1.9657,  ..., -2.0007, -2.0182, -2.0357]],

        [[-1.5256, -1.5256, -1.5256,  ..., -1.7173, -1.2816, -0.5321],
         [-1.5256, -1.5256, -1.5256,  ..., -1

tensor([[[-1.8953, -1.9295, -1.9295,  ..., -1.9295, -1.6727, -1.3302],
         [-1.9124, -1.9124, -1.9295,  ..., -1.8953, -1.6213, -1.2103],
         [-1.9295, -1.9295, -1.9295,  ..., -1.8953, -1.5870, -1.1932],
         ...,
         [-2.1008, -2.1179, -2.1179,  ...,  0.3994,  0.1597,  0.0227],
         [-2.1179, -2.1179, -2.1179,  ...,  0.3994,  0.1768,  0.0741],
         [-2.1179, -2.1179, -2.1179,  ...,  0.3652,  0.1939,  0.0741]],

        [[-1.8081, -1.8256, -1.8256,  ..., -1.8256, -1.5630, -1.1779],
         [-1.8256, -1.8256, -1.8431,  ..., -1.8256, -1.5105, -1.0903],
         [-1.8431, -1.8431, -1.8431,  ..., -1.8606, -1.5105, -1.1078],
         ...,
         [-2.0182, -2.0007, -2.0007,  ..., -1.9657, -2.0007, -2.0357],
         [-2.0357, -2.0182, -2.0182,  ..., -1.9657, -1.9832, -1.9657],
         [-2.0357, -2.0182, -2.0182,  ..., -2.0007, -2.0182, -2.0007]],

        [[-1.5430, -1.5604, -1.5604,  ..., -1.6999, -1.4384, -1.0724],
         [-1.5604, -1.5604, -1.5779,  ..., -1

tensor([[[-2.0152, -2.0152, -1.9809,  ..., -0.7137, -1.0562, -1.3130],
         [-2.0152, -2.0152, -1.9809,  ..., -0.5938, -0.9534, -1.2445],
         [-2.0152, -1.9980, -1.9809,  ..., -0.4054, -0.7993, -1.1247],
         ...,
         [-1.7412, -1.6555, -1.4329,  ...,  0.6734,  0.7762,  0.8618],
         [-1.7754, -1.7240, -1.5357,  ...,  0.6221,  0.7419,  0.8447],
         [-1.8610, -1.7925, -1.7069,  ...,  0.5193,  0.6563,  0.7419]],

        [[-1.9307, -1.9307, -1.9132,  ..., -1.1253, -1.4230, -1.6681],
         [-1.9307, -1.9307, -1.9132,  ..., -1.0028, -1.3179, -1.5805],
         [-1.9307, -1.9307, -1.9132,  ..., -0.8102, -1.1779, -1.4755],
         ...,
         [-2.0357, -2.0182, -1.8256,  ..., -2.0357, -2.0007, -1.9132],
         [-2.0357, -2.0357, -1.9132,  ..., -2.0357, -2.0182, -1.9132],
         [-2.0182, -2.0182, -1.9482,  ..., -2.0357, -1.9832, -1.9482]],

        [[-1.6999, -1.6999, -1.7173,  ..., -1.0376, -1.3164, -1.5604],
         [-1.6999, -1.7347, -1.7173,  ..., -0

tensor([[[-1.6555, -1.6555, -1.6555,  ..., -1.9295, -1.9295, -1.9467],
         [-1.5528, -1.6555, -1.7412,  ..., -1.9295, -1.9467, -1.9467],
         [-1.3987, -1.6898, -1.8097,  ..., -1.9467, -1.9467, -1.9467],
         ...,
         [ 1.6153,  1.9235,  2.2489,  ..., -1.9638, -2.0494, -2.0665],
         [ 1.4440,  1.7694,  2.2318,  ..., -1.9809, -2.0494, -2.0494],
         [ 1.4098,  1.7523,  2.1119,  ..., -1.9809, -1.9980, -1.9809]],

        [[-1.5455, -1.5455, -1.5805,  ..., -1.8081, -1.8081, -1.8256],
         [-1.4405, -1.5805, -1.6681,  ..., -1.8081, -1.8256, -1.8256],
         [-1.3179, -1.6155, -1.7731,  ..., -1.8256, -1.8256, -1.8256],
         ...,
         [-1.8957, -1.6155, -1.2304,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -1.7731, -1.3179,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -1.7906, -1.4580,  ..., -2.0357, -2.0182, -2.0007]],

        [[-1.4210, -1.4210, -1.4384,  ..., -1.5953, -1.5953, -1.6127],
         [-1.3164, -1.4384, -1.5256,  ..., -1

tensor([[[-1.7925, -1.7925, -1.7754,  ..., -1.9467, -1.9295, -1.8953],
         [-1.7925, -1.7412, -1.7240,  ..., -1.9467, -1.9295, -1.8953],
         [-1.7240, -1.6555, -1.6042,  ..., -1.9467, -1.9467, -1.9295],
         ...,
         [ 2.1804,  2.0092,  1.5297,  ..., -1.9467, -1.9638, -1.9980],
         [ 2.2147,  2.1462,  1.7523,  ..., -1.9809, -1.9638, -1.9980],
         [ 2.2489,  2.2318,  1.9920,  ..., -1.9809, -1.9638, -2.0323]],

        [[-1.7206, -1.7206, -1.7031,  ..., -1.8606, -1.8782, -1.8957],
         [-1.7731, -1.7206, -1.6506,  ..., -1.8606, -1.8782, -1.8957],
         [-1.7031, -1.6331, -1.5805,  ..., -1.8606, -1.8782, -1.8782],
         ...,
         [-1.3004, -1.5105, -2.0007,  ..., -2.0357, -2.0357, -2.0357],
         [-1.3004, -1.3704, -1.8256,  ..., -2.0357, -2.0357, -2.0357],
         [-1.2654, -1.2654, -1.5805,  ..., -2.0357, -2.0357, -2.0357]],

        [[-1.5604, -1.5604, -1.5430,  ..., -1.5953, -1.5604, -1.5604],
         [-1.5953, -1.5430, -1.4907,  ..., -1

...

tensor([[[-0.1828, -1.7412, -2.0665,  ..., -1.9980, -1.9980, -2.0323],
         [-1.5357, -1.9980, -2.0665,  ..., -1.9124, -1.9980, -1.9809],
         [-2.0837, -2.1008, -2.0665,  ..., -1.5185, -1.8097, -1.9638],
         ...,
         [ 0.9474,  1.2043,  1.2557,  ..., -1.0048, -0.9363, -0.9705],
         [ 1.1700,  1.1700,  1.1358,  ..., -0.8164, -0.7822, -0.8164],
         [ 0.3309,  0.3138,  0.7591,  ..., -0.7993, -0.8335, -0.8678]],

        [[-0.0749, -1.6506, -2.0007,  ..., -2.0357, -2.0182, -2.0182],
         [-1.4580, -1.9132, -2.0007,  ..., -1.9832, -2.0357, -2.0182],
         [-2.0182, -2.0357, -2.0007,  ..., -1.6506, -1.9132, -2.0182],
         ...,
         [ 0.4328,  0.7129,  0.9405,  ..., -1.4580, -1.3354, -1.3529],
         [ 0.6604,  0.6954,  0.8179,  ..., -1.2829, -1.1954, -1.1954],
         [-0.1975, -0.1800,  0.4328,  ..., -1.2829, -1.2654, -1.2654]],

        [[ 0.2871, -1.2990, -1.7173,  ..., -1.8044, -1.8044, -1.8044],
         [-1.0898, -1.5779, -1.7347,  ..., -1

tensor([[[-2.1008, -2.0665, -2.0323,  ..., -1.7069, -1.8268, -1.9295],
         [-2.1008, -2.0837, -2.0665,  ..., -1.3815, -1.3644, -1.5014],
         [-2.1008, -2.0494, -2.0494,  ..., -1.1075, -0.7993, -0.8678],
         ...,
         [ 1.4783,  0.9474,  0.5364,  ...,  0.5707,  0.6392,  0.2624],
         [ 1.2043,  0.7591,  0.4679,  ...,  1.1700,  0.7077, -0.0287],
         [ 1.1015,  0.6734,  0.4337,  ...,  1.3584,  0.6906, -0.0801]],

        [[-2.0182, -2.0007, -2.0007,  ..., -1.7556, -1.8782, -1.9832],
         [-2.0182, -2.0182, -2.0357,  ..., -1.4405, -1.4055, -1.5455],
         [-2.0357, -2.0007, -2.0182,  ..., -1.1604, -0.8452, -0.8978],
         ...,
         [ 0.7479,  0.2402, -0.1800,  ..., -0.0749, -0.0049, -0.4076],
         [ 0.4853,  0.0476, -0.2325,  ...,  0.5553,  0.0826, -0.6877],
         [ 0.3978, -0.0224, -0.2675,  ...,  0.7654,  0.0826, -0.7052]],

        [[-1.7522, -1.7347, -1.7347,  ..., -1.5779, -1.6999, -1.7870],
         [-1.7522, -1.7522, -1.7870,  ..., -1

tensor([[[-1.4158, -1.4672, -1.6213,  ...,  1.4098,  1.3755,  1.3242],
         [-1.5699, -1.7583, -1.8782,  ...,  1.4098,  1.3584,  1.3413],
         [-1.9124, -1.9467, -1.9467,  ...,  1.4612,  1.3584,  1.3584],
         ...,
         [ 0.2796,  0.3994, -0.2856,  ..., -0.3198,  0.3652, -0.1657],
         [ 0.2796,  0.4679, -0.1314,  ..., -0.1143,  0.4337,  0.0227],
         [ 0.2967,  0.5536,  0.0912,  ...,  0.4508,  0.6734,  0.3481]],

        [[-1.4230, -1.5105, -1.6506,  ...,  1.4832,  1.4132,  1.3606],
         [-1.5805, -1.8081, -1.9307,  ...,  1.4832,  1.3957,  1.3782],
         [-1.8957, -2.0007, -2.0007,  ...,  1.5357,  1.3957,  1.3957],
         ...,
         [-0.3725, -0.2850, -1.0203,  ..., -0.4951,  0.2577, -0.2500],
         [-0.3550, -0.2150, -0.8452,  ..., -0.3025,  0.2927, -0.1275],
         [-0.3200, -0.1275, -0.6001,  ...,  0.2402,  0.4853,  0.1702]],

        [[-1.2467, -1.3164, -1.4733,  ...,  1.6291,  1.5420,  1.4722],
         [-1.3861, -1.6127, -1.7347,  ...,  1

tensor([[[-2.0323, -1.8953, -1.0562,  ..., -1.6042, -1.6898, -1.6384],
         [-1.9295, -1.4329,  0.0912,  ..., -1.6727, -1.7069, -1.6727],
         [-1.5870, -0.4054,  1.1015,  ..., -1.6898, -1.6727, -1.6042],
         ...,
         [ 0.8276,  0.8276,  0.7591,  ..., -1.4843, -1.4672, -1.0904],
         [ 0.4508,  0.4851,  0.3823,  ..., -1.4672, -1.2788, -0.7308],
         [ 0.2453,  0.2967,  0.2282,  ..., -1.3473, -0.9534, -0.2342]],

        [[-2.0182, -1.9307, -1.1253,  ..., -1.9657, -1.9832, -1.9132],
         [-1.9482, -1.4755,  0.0301,  ..., -2.0182, -2.0182, -1.9482],
         [-1.6331, -0.4426,  1.0630,  ..., -2.0357, -2.0007, -1.8957],
         ...,
         [ 0.2402,  0.2752,  0.2927,  ..., -2.0357, -1.9657, -1.5805],
         [-0.1975, -0.1099, -0.1099,  ..., -2.0182, -1.8606, -1.2654],
         [-0.4076, -0.3025, -0.2850,  ..., -2.0007, -1.5630, -0.7927]],

        [[-1.8044, -1.7173, -0.9853,  ..., -1.6824, -1.6999, -1.6476],
         [-1.7522, -1.2990,  0.1651,  ..., -1

tensor([[[-1.6042, -1.4500, -1.1760,  ..., -1.2617, -1.6042, -1.7412],
         [-1.5870, -1.4158, -1.1247,  ..., -1.0733, -1.5357, -1.7069],
         [-1.5528, -1.4158, -1.0733,  ..., -0.8507, -1.4672, -1.7240],
         ...,
         [ 0.8789,  0.6392, -0.0116,  ...,  0.5707,  0.3138, -0.0116],
         [ 0.8447,  0.6906,  0.1597,  ...,  0.7077,  0.3823, -0.0458],
         [ 0.8104,  0.7248,  0.4337,  ...,  0.7077,  0.3652, -0.0287]],

        [[-1.9482, -1.8957, -1.6681,  ..., -1.7381, -2.0007, -2.0357],
         [-1.9307, -1.8782, -1.6331,  ..., -1.5630, -1.9482, -2.0357],
         [-1.8957, -1.8782, -1.5805,  ..., -1.3354, -1.8782, -2.0357],
         ...,
         [ 0.2577,  0.0126, -0.6527,  ...,  0.4853,  0.2402, -0.0749],
         [ 0.2227,  0.0826, -0.4776,  ...,  0.6254,  0.3102, -0.0924],
         [ 0.1877,  0.1176, -0.1800,  ...,  0.6429,  0.3102, -0.0749]],

        [[-1.7870, -1.7173, -1.4733,  ..., -1.4559, -1.7522, -1.8044],
         [-1.7696, -1.6824, -1.4384,  ..., -1

In [ ]:
# aligned = []
# names = []
# for x, y in loader:
#     print(x)
#     print(y)
#     x_aligned, prob = mtcnn(x, return_prob=True)
#     if x_aligned is not None:
#         print('Face detected with probability: {:8f}'.format(prob))
#         aligned.append(x_aligned)
#         names.append(dataset.idx_to_class[y])

In [ ]:
video = mmcv.VideoReader('trimmed.mp4')
frames = list()

for idx, frame in enumerate(video):
    if idx % 30 == 0:
        frames.append(Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)))

print(len(frames))

In [ ]:
frames_tracked = []
for idx_frame, frame in enumerate(frames):
#     print(f'Tracking frame: {idx_frame}')
    
    # Detect faces
    boxes, _ = mtcnn.detect(frame)
    
    # Draw faces
    frame_draw = frame.copy()
    draw = ImageDraw.Draw(frame_draw)
    for box in boxes:
        draw.rectangle(box.tolist(), outline=(255, 0, 0), width=6)
    
    # Add to frame list
    final_frame = frame_draw.resize((1280, 720), Image.BILINEAR)
    
    frames_tracked.append(final_frame)
    
    display.display(final_frame)
    
print('Done')